In [ ]:
%%Capture
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install sklearn
!pip install scipy
!pip install statsmodels
!pip install xgboost
!pip install lightgbm
!pip install google-colab


# Importing libraries


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy
import statsmodels
import xgboost
import lightgbm


# load the required data
data = pd.read_excel('../data/michaelgoboola.xlsx', sheet_name='data')
data.head()

In [ ]:
# load data from  google drive
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/michaelgoboola.xlsx', sheet_name='data') # change path to your path
data.head()

# Describe the data

In [ ]:
data.describe()

# Plot and Describe the data


In [ ]:
data.hist(figsize=(20,20))
plt.show()